In [ ]:
library(data.table)
library(rvest)
library(magrittr)
library(stringr)
library(ggplot2)

In [ ]:
# define the URL we want to scrap
url <- "https://www.statistik.at/web_de/statistiken/wirtschaft/preise/verbraucherpreisindex_vpi_hvpi/zeitreihen_und_verkettungen/025206.html"

In [ ]:
# read the whole webpage-content to a variable
webContent <- read_html(url)

In [ ]:
# we know that commas are used for the decimal mark.
# Even if we use the option dec = "." we get as data
# type character because the missing values are indicated
# with a dot "."
vpiTable <- html_table(webContent)[[1]] %>% setDT()
vpiTable

In [ ]:
# the last row doesn't contain any relevant information so we delete it 
vpiTable <- vpiTable[-.N, ]
vpiTable

In [ ]:
# Replace all dots with NA's and cast the remaining values to numeric
for(j in 2:ncol(vpiTable)){
  # change all dots do NA's
  set(vpiTable, i = which(vpiTable[[j]] == "."), j = j, value = NA)
  # convert text to numbers
  set(vpiTable, j = j, value = as.numeric(sub(",", ".", vpiTable[[j]], fixed = TRUE)))
}
vpiTable

In [ ]:
# Find all average values
vpiRaw <- vpiTable[str_detect(vpiTable$Monat, "^[^A-Z]"), ]
vpiRaw

In [ ]:
# define column "Monat" as factor otherwise plotting would be in alphabetical order
vpiRaw$Monat <- factor(vpiRaw$Monat, levels=unique(vpiRaw$Monat))

# plot column "Monat" against column "KHPI" (Kleinhandelspreisindex)
ggplot(data = vpiRaw, aes(x = Monat , y = KHPI, group = 1)) +
  geom_line()+
  geom_point()